In [12]:
import polars as pl
from pathlib import Path

# stats generated from ~1300 replays from 1/1/2023 - 4/30/2023
take_hit = pl.read_parquet(Path(R"./Output/take_hits.parquet"))
pl.Config.set_tbl_rows(-1)
take_hit.schema

{'date_time': Datetime(time_unit='us', time_zone='America/Chicago'),
 'slippi_version': Utf8,
 'match_id': Utf8,
 'match_type': Utf8,
 'game_number': Int64,
 'stage': Utf8,
 'duration': Duration(time_unit='ms'),
 'result': Utf8,
 'port': Utf8,
 'connect_code': Utf8,
 'character': Utf8,
 'costume': Utf8,
 'opnt_character': Utf8,
 'frame_index': Int64,
 'stocks_remaining': Int64,
 'grounded': Boolean,
 'percent': Float64,
 'last_hit_by': Utf8,
 'state_before_hit': Utf8,
 'crouch_cancel': Boolean,
 'hitlag_frames': Int64,
 'stick_regions_during_hitlag': List(Utf8),
 'sdi_inputs': List(Utf8),
 'asdi': Utf8,
 'start_pos': List(Float64),
 'end_pos': List(Float64),
 'kb_angle': Float64,
 'di_stick_pos': List(Float64),
 'di_efficacy': Float64,
 'final_kb_angle': Float64,
 'kb_velocity': List(Float64),
 'final_kb_velocity': List(Float64)}

In this section, I'm going to restrict it to Falco vs Fox to help keep the output understandable, but know that this could be applied to any character or group of characters

In [13]:
fox_moves = (take_hit.filter((pl.col("character") == "FALCO") & (pl.col("opnt_character") == "FOX"))
    .groupby(pl.col("last_hit_by"))
    .agg([pl.count()])
    .sort(pl.col("count"), descending=True)
)
print(fox_moves)

shape: (26, 2)
┌──────────────────────────┬───────┐
│ last_hit_by              ┆ count │
│ ---                      ┆ ---   │
│ str                      ┆ u32   │
╞══════════════════════════╪═══════╡
│ DOWN_SPECIAL             ┆ 1758  │
│ BAIR                     ┆ 1704  │
│ DAIR                     ┆ 1613  │
│ NAIR                     ┆ 1088  │
│ UAIR                     ┆ 623   │
│ UP_SMASH                 ┆ 586   │
│ UP_TILT                  ┆ 482   │
│ JAB_1                    ┆ 363   │
│ DASH_ATTACK              ┆ 352   │
│ UP_SPECIAL               ┆ 162   │
│ PUMMEL                   ┆ 151   │
│ FAIR                     ┆ 142   │
│ SIDE_SPECIAL             ┆ 124   │
│ NEUTRAL_SPECIAL          ┆ 107   │
│ DOWN_TILT                ┆ 76    │
│ SIDE_TILT                ┆ 68    │
│ JAB_2                    ┆ 66    │
│ SIDE_SMASH               ┆ 61    │
│ FORWARD_THROW            ┆ 53    │
│ GET_UP_ATTACK_FROM_BACK  ┆ 45    │
│ GET_UP_ATTACK_FROM_FRONT ┆ 27    │
│ DOWN_SMASH           

That's a lot of information. It also highlights a small bug in the parsing logic, which is fox's up throw being counted inaccurately. 

In [17]:
_moves = (take_hit.filter((pl.col("character") == "FALCO") & (pl.col("opnt_character") == "PEACH"))
    .groupby(pl.col("last_hit_by"))
    .agg([pl.count()])
    .sort(pl.col("count"), descending=True)
)
print(_moves)

shape: (24, 2)
┌─────────────────────────┬───────┐
│ last_hit_by             ┆ count │
│ ---                     ┆ ---   │
│ str                     ┆ u32   │
╞═════════════════════════╪═══════╡
│ NAIR                    ┆ 177   │
│ BAIR                    ┆ 117   │
│ DOWN_SMASH              ┆ 106   │
│ DAIR                    ┆ 98    │
│ DASH_ATTACK             ┆ 87    │
│ UP_SPECIAL              ┆ 74    │
│ UP_THROW                ┆ 56    │
│ PUMMEL                  ┆ 55    │
│ FAIR                    ┆ 44    │
│ UAIR                    ┆ 43    │
│ JAB_1                   ┆ 41    │
│ NON_STALING             ┆ 39    │
│ JAB_2                   ┆ 30    │
│ FORWARD_THROW           ┆ 25    │
│ NEUTRAL_SPECIAL         ┆ 23    │
│ BACK_THROW              ┆ 18    │
│ GET_UP_ATTACK_FROM_BACK ┆ 7     │
│ OPEN_PARASOL            ┆ 7     │
│ SIDE_TILT               ┆ 7     │
│ UP_SMASH                ┆ 3     │
│ LEDGE_GET_UP_ATTACK     ┆ 3     │
│ LEDGE_GET_UP_ATTACK_100 ┆ 2     │
│ SIDE_SMASH 